I want a riddance of backtrader!!!

In [70]:
import os
import pandas as pd
import numpy as np
from params import *
from utils.read2df import read2df
from utils.cointncorr import CointnCorr

In [10]:
'''
Load data from pickle or recalculate
'''

import pickle

# If there is already a pickle file
pickle_file = 'result/cointncorr.pickle'

if os.path.exists(pickle_file):
    with open('result/cointncorr.pickle', 'rb') as pk:
        data = pickle.load(pk)

    pair, best_freq = data 
    best_pair = f"{pair[0]}_{pair[1]}"

    print("===========================================")
    print(f"Data loaded as: {best_pair} under {best_freq} interval")
    print("===========================================")

else:
    raise Error("No pickle file found")    

Data loaded as: BTCBUSD_BTCUSDC under 1m interval


In [20]:
'''
Read the downloaded OHLCV data into `pandas` dataframe
'''

# dfs = read2df(symbols, freqs)
dfs = read2df(symbols=data[0], freqs={data[1]: freqs[data[1]]})

df0 = dfs[0][dfs[0]['tic']==data[0][0]][['time', 'datetime', 'tic', 'close']].reset_index(drop=True)
df1 = dfs[0][dfs[0]['tic']==data[0][1]][['time', 'datetime', 'tic', 'close']].reset_index(drop=True)

df0

,time,datetime,tic,close
0,1568887379999,2019-09-19 10:02:59.999,BTCBUSD,9881.43
1,1568887439999,2019-09-19 10:03:59.999,BTCBUSD,9881.43
2,1568887499999,2019-09-19 10:04:59.999,BTCBUSD,9878.36
3,1568887559999,2019-09-19 10:05:59.999,BTCBUSD,9878.36
4,1568887619999,2019-09-19 10:06:59.999,BTCBUSD,9878.36
...,...,...,...,...
1926187,1698796559999,2023-10-31 23:55:59.999,BTCBUSD,34654.72
1926188,1698796619999,2023-10-31 23:56:59.999,BTCBUSD,34644.00
1926189,1698796679999,2023-10-31 23:57:59.999,BTCBUSD,34661.64
1926190,1698796739999,2023-10-31 23:58:59.999,BTCBUSD,34659.99


In [68]:
'''
Calculate the distance
'''

df_distance = df0.assign(distance=df0['close'] - df1['close'])[['time', 'datetime', 'distance']]
df_distance.head(10)

,time,datetime,distance
0,1568887379999,2019-09-19 10:02:59.999,3.54
1,1568887439999,2019-09-19 10:03:59.999,5.75
2,1568887499999,2019-09-19 10:04:59.999,6.77
3,1568887559999,2019-09-19 10:05:59.999,9.00
4,1568887619999,2019-09-19 10:06:59.999,10.30
5,1568887679999,2019-09-19 10:07:59.999,6.48
6,1568887739999,2019-09-19 10:08:59.999,10.18
7,1568887799999,2019-09-19 10:09:59.999,10.02
8,1568887859999,2019-09-19 10:10:59.999,10.02
9,1568887919999,2019-09-19 10:11:59.999,18.41


In [78]:
# It's too slow...
# Guess the reason to use backtrader is for speed

PERIOD = 100
OPEN_THRE = 2.0
CLOS_THRE = 0.1

df_trade = pd.DataFrame(columns=['zscore', 'open_thre', 'clos_thre'])

for i in range(PERIOD, len(df_distance)):
    df_iter = df_distance[i-PERIOD: i]['distance']

    stddev = np.std(df_iter)
    df_zscore = (df_iter-np.mean(df_iter))/stddev
    last_zscore = df_zscore.iloc[-1]

    compare_open_thre= 1 if last_zscore >= OPEN_THRE * stddev else -1 if last_zscore <= -OPEN_THRE * stddev else 0
    compare_clos_thre = 1 if last_zscore >= CLOS_THRE * stddev else -1 if last_zscore <= -CLOS_THRE * stddev else 0

    df_trade = pd.concat([df_trade, pd.DataFrame([{'zscore': df_zscore.iloc[-1], 'open_thre': compare_open_thre, 'clos_thre': compare_clos_thre}])], ignore_index=True)

100